## Aula 3 - Pandas e Fontes de dados

### Manipulação de Fontes de Dados

In [ ]:
#Fazer o download dos datasets
!git clone https://github.com/alexlopespereira/curso_ciencia_dados2020.git ../../curso_ciencia_dados2020

In [ ]:
import pandas as pd
import numpy as np

#### Carregar dados de um Arquivo XLS

In [ ]:
### Ler o arquivo com o pib dos municípios brasileiros e guarde numa variável df_pib
df_pib = pd.read_excel('../../curso_ciencia_dados2020/data/originais/pib/pib_municipios.xlsx', sheet_name='Tabela') 
# sheet_name é o argumento para o aba da planilha desejada. 
# Se você quiser a 1a aba, não precisa especificar.




In [ ]:
# Mostra os primeiros 4 registros da variavel df_pib
df_pib.head(4)
### As 3 primeiras e a ultima linha são inuteis

In [ ]:
# Mostra os ultimos 4 registros da variavel df_pib
df_pib.tail(4)

In [ ]:
# Ler o arquivo do pib removendo as 3 primeiras linhas e a última
df_pib = pd.read_excel('../../curso_ciencia_dados2020/data/originais/pib/pib_municipios.xlsx', skiprows=3, skipfooter=1)
# Mostra os primeiros registros
df_pib.head()
# O nome das 3 primeiras colunas está incoerente
'latin_1'
'utf8'
'utf_8'

In [ ]:
# Renomear estas colunas
df_pib.rename(columns={'Unnamed: 0': 'nivel', 'Unnamed: 1': 'cod_ibge7', 
                      'Unnamed: 2': 'municipio'}, inplace=True)
# Mostra os primeiros registros
df_pib.head()

#### Carregar dados de um Arquivo CSV

In [ ]:
# Salvar este dataframe em formato CSV
# O separador padrão é vírgula. Use o ; para testar a especificação de um separador
# Recomendação para nomes de arquivos: use nomes simples, sem espaço e sem caracteres especiais.
df_pib.to_csv('../../curso_ciencia_dados2020/data/processados/pib/pib_municipios.csv', sep=";", index=False)
# Para mostrar as primeiras linhas do arquivo no Linux: !head ~/work/curso_ciencia_dados2020/data/pib_municipios.csv

In [ ]:
# Carregue o arquivo pib_municipios.csv. Atente para o separador correto.
df_pib_csv = pd.read_csv('../../curso_ciencia_dados2020/data/processados/pib/pib_municipios.csv', sep=";")
# Mostra os primeiros registros


#### Carregar dados de uma API REST e Arquivo JSON

In [ ]:
# Ler um arquivo JSON de uma URL para um dataframe df_dolar. 
# Pode-se carregar de uma pasta local também.
url = "https://api.exchangerate-api.com/v4/latest/USD"
df_dolar = pd.read_json(url)

In [ ]:
# Mostra os primeiros registros deste dataframe df_dolar
df_dolar.head()

In [ ]:
# Teste a conversão para DataFrame a partir de um JSON mais complexo 
# Carregue o dataframe retornado pela chamada 
#   GET à URL "https://www.servicos.gov.br/api/v1/servicos/9029"
url_servicos = "https://www.servicos.gov.br/api/v1/servicos/9029"
df_servicos = pd.read_json(url_servicos)
# Mostra os primeiros registros
df_servicos.head()
# O resultado não foi como esperado

#### A função read_json não conseguiu processar e criar o DataFrame esperado a partir do JSON
#### A solução alternativa é selecionar os atributos do JSON e colocar num dicionário para construir um DataFrame

In [ ]:
import requests # Biblioteca que realiza requisições HTTP em python

def get_servicos(siorg):
    '''Retorna os dados da API de serviços para o órgao especificado pelo argumento siorg
    '''
    url = 'https://servicos.gov.br/api/v1/servicos/orgao/{0}'.format(siorg)
    response = requests.get(url) # Realiza uma requisição http
    input_json = response.json()
    if not input_json or 'resposta' not in input_json:
        print("Os dados da api de servicos nao estao disponiveis")
        return {}
    input_json = input_json['resposta']
    result = []
    for i in input_json: # iteração para cada serviço do órgão
        servico_id = i['id'].split('/')[6]
        servico_nome = i['nome']
        servico_url = i['url'][36:]
        orgao_id = i['orgao']['id'].split('/')[5]
        orgao_dbId = i['orgao']['dbId']
        orgao_nome = i['orgao']['nomeOrgao']
        result.append({'servico_id': servico_id, 'servico_nome': servico_nome, 'servico_url': servico_url, 'orgao_nome': orgao_nome,
                              'orgao_id': orgao_id, 'orgao_dbId': orgao_dbId})
    return result


In [ ]:
# Testa o método com o sirgo da Fio Cruz
d = get_servicos(315) 
# Transforma a lista de dicionários num DataFrame
df_servicos = pd.DataFrame(d) 
# # Mostra os primeiros registros
df_servicos.head()

#### Carregar dados de Arquivo HTML (Web Scrapy)

In [ ]:
# Ler um arquivo HTML (https://en.wikipedia.org/wiki/Pythonidae) 
#   carregando as tabelas presentes (Web Scrapy)
df_python = pd.read_html('https://en.wikipedia.org/wiki/Pythonidae')
# len(df_python)
# Mostra os primeiros registros da tabela 2 da página analisada
df_python[2].head()

In [ ]:
# Renomear as colunas do dataframe retirando colchetes e espaço, usando
#     um mapeamento 'antes':'depois' no formato de um dicionário.
# Use inplace=True
df_python[2].rename(columns={'Taxon author[2]': 'taxon_author', 'Species[2]': 'species', 
                      'Subsp.[a][2]': 'subspecies', 'Common name': 'common_name', 
                      'Geographic range[1]': 'geographic_range'}, inplace=True)
# Mostra os primeiros registros da tabela 2
df_python[2].head()

#### Dados em Bancos de Dados

##### Converter um dataframe para uma tabela de banco de dados

In [ ]:
# O banco de dados SQLite é um banco bem simples gerenciado em um único arquivo. 
# É usado, por exemplo, em aplicativos de celular.
# A biblioteca sqlalchemy é um mapeamento objeto-relacional para vários SGBDs.
# Esta biblioteca consegue executar comandos SQL em todos os SGBDs compatíveis.
import sqlalchemy as sqla

# Crie uma conexão com o banco de dados
con = sqla.create_engine('sqlite:///../../curso_ciencia_dados2020/data/processados/python/python.sqlite')

# Escreva o DataFrame df_python[2] numa tabela de um banco de dados SQLite
df_python[2].to_sql("python", con, if_exists="replace")

#### Você pode visualizar seu banco de dados de forma independente 
#### em https://inloop.github.io/sqlite-viewer/

##### Carregar os dados de uma tabela de banco de dados para um DataFrame

In [ ]:
# Execute uma query SELECT no banco de dados sqlite convertendo toda a tabela python num DataFrame
# ajustando o parametro index_col='index'
df_sqlite = pd.read_sql_query("SELECT * from python WHERE subspecies > 0", con, index_col='index')

# Visualiza os primeiros resultados
df_sqlite.head()

##### Fazer cálculos envolvendo agregação

In [ ]:
# Calcular a soma quantidade de subespecies de cada especie
df_sqlite_agg = pd.read_sql_query("SELECT common_name, SUM(subspecies) \
                                    FROM python \
                                    GROUP BY species", con)
# Mostra os primeiros registros da agregação
df_sqlite_agg.head()

In [ ]:
# Calcular a quantidade de subspecies de cada região geográfica e ordenar pela 
# quantidade de subspecies de forma decrescente
df_sqlite_agg2 = pd.read_sql_query("SELECT geographic_range, SUM(species) AS n_species \
                                       FROM python \
                                       GROUP BY geographic_range \
                                       ORDER BY n_species DESC", con)
# Mostra os primeiros 10 registros da agregação
df_sqlite_agg2.head(10)


### Outras Operacoes no Pandas

#### Descartando valores faltantes (NA ou NaN)

In [ ]:
from numpy import nan as NA
# Considere a serie a seguir
data = pd.Series([1, NA, 3.5, NA, 7])
# Remova os valores NA
data.dropna()

In [ ]:
# Comando equivalente ao dropna, mas usando o método notnull()
data[data.notnull()]

#### Preenchendo valores faltantes

In [ ]:
# Considere o seguinte DataFrame
# Construir um dataframe a partir de uma matriz de 7 x 3 de números aleatórios 
# de uma distribuição normal padrão
df = pd.DataFrame(np.random.randn(7, 3)) 
# Preenchendo o DataFrame com alguns valores NA
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

In [ ]:
# Preencha os valores NA com zero
df.fillna(0)

In [ ]:
# Use um mapeamento (com dicionário) para preencher os valores NA
# Na coluna 1 substitua NA por 0.5 e na coluna 2 substitua NA por 0
df.fillna({1: 0.5, 2: 0})

In [ ]:
# Preencha com zero alterando o DataFrame df
df.fillna(0, inplace=True)
df

In [ ]:
# Considere o seguinte dataframe
df = pd.DataFrame(np.random.randn(7, 3)) 
# Preencha o DataFrame com alguns valores NA
df.iloc[1:4, 1] = NA
df.iloc[1:3, 2] = NA
df

In [ ]:
# Preencha os valores NA com o método ffill 
df.fillna(method='ffill')

### Remover duplicatas

In [ ]:
# Considere o seguinte dataframe
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
# Mostre quais desses itens são duplicados
data.duplicated()

In [ ]:
# Remova os itens duplicados
data.drop_duplicates()

### Indexação Hierárquica

#### Possibilita mais de um nível de indexação num eixo

In [ ]:
# Considere a seguinte Serie
data = pd.Series(np.random.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                                            [1, 2, 3, 1, 3, 1, 2, 2, 3]])
# Mostre o indice hierárquico
data.index

In [ ]:
# Faça um filtro com uma lista
data.loc[['b', 'd']]

In [ ]:
# Faça um filtro no 2o Nível (mais interno)
data.loc[:, 2]

In [ ]:
# Considere o seguinte dataframe
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

In [ ]:
# Resumo estatístico por nível
# Extraia a soma da agregação do nível 2 (mais interno)
frame.sum(level='key2')

### merge (fundir/juntar)
#### A chave de junção (identificador único) foi inferida a partir do contexto (procurando nas colunas)
#### Também pode ser especificada com o argumento on (Ex.: on='key')

In [ ]:
# Considere os seguintes DataFrames
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [ ]:
# Mostre o dataframe df1
df1

In [ ]:
# Mostre o dataframe df2
df2

In [ ]:
# Faça o merge dos dois dataframes usando como chave de junção a coluna 'key'
# O parametro on neste caso é redundante. Funcionaria sem ele.
pd.merge(df1, df2, on='key')

# Por que chamamos esse tipo de merge/join de inner ? R.: Porque ele considera apenas a 
#     intersecção dos dois conjuntos de chaves.
# O argumento how='inner' é o padrão.

In [ ]:
# Faça também o merge com o argumento how='outer'
pd.merge(df1, df2, how='outer')

### join (fundir/juntar)
#### Semelhante ao merge, mas a chave de junção é o índice do DataFrame

In [ ]:
# Considere os seguintes DataFrames
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'], 
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],index=['b', 'c', 'd', 'e'],
                     columns=['Missouri', 'Alabama'])


In [ ]:
# Mostre o dataframe left2
left2

In [ ]:
# Mostre o dataframe right2
right2

In [ ]:
# Faça o join entre os dois DataFrames sem descartar registros que não estejam 
#      nos dois DataFrames
left2.join(right2, how='outer')

In [ ]:
# Considerando os dataframes left2 e right2 definidos acima
# Faça o join entre os dois DataFrames sem descartar registros que estejam 
# apenas no dataframe left2, e descartando registros que estejam apenas no right2
left2.join(right2, how='left')

### Reshaping / Pivoting (Pivotar)

In [ ]:
# Considere o seguinte DataFrame
table = {
    'Aluno': ['AlunoA', 'AlunoA', 'AlunoA', 'AlunoA', 'AlunoB', 'AlunoB', 'AlunoB', 'AlunoB'],
    'Disciplina': ['Portugues', 'Matematica', 'Geografia', 'História', 'Portugues', 'Matematica', 'Geografia', 'História'],
    'Objetiva': [8.5, 7.5, 9, 10, 8.5, 7.5, 9, 10],
    'Discursiva': [6, 6.5, 7.5, 7, 8.5, 7.5, 9, 10]}
df_provas = pd.DataFrame(table)
df_provas

In [ ]:
# Pivote o dataframe df_provas colocando a coluna Aluno como índice, 
# os valores da coluna Disciplina como colunas, e os valores da coluna Objetiva
# como conteúdo do novo dataframe
df_pivoted = df_provas.pivot(index='Aluno', columns='Disciplina', values=['Objetiva', 'Discursiva'])
df_pivoted

In [ ]:
df_pivoted['Objetiva'].loc['AlunoA',['Geografia', 'História']]

### E quando houver valores repetidos ?
#### Pivotar com o mesmo método pivot() gera exceção. Neste caso, use o método pivot_table 
#### mean é a métrica padrão de cálculo sobre a de agregação

In [ ]:
# Considere o seguinte DataFrame
table2 = {
    'Aluno': ['AlunoA', 'AlunoA', 'AlunoA', 'AlunoA', 'AlunoA', 'AlunoB', 'AlunoB', 'AlunoB', 'AlunoB'],
    'Disciplina': ['Portugues', 'Matematica', 'Geografia', 'Geografia', 'História', 'Portugues', 'Matematica', 'Geografia', 'História'],
    'Objetiva': [8.5, 7.5, 9, 10, 9, 8.5, 7.5, 9, 10],
    'Discursiva': [6, 6.5, 7.5, 7, 8, 8.5, 7.5, 9, 10]}
df_provas2 = pd.DataFrame(table2)
df_provas2

In [ ]:
# Pivotar com o mesmo comando gera uma exceção
# df_pivoted2 = df_provas2.pivot(index='Aluno', columns='Disciplina', values='Objetiva')
# Use a funcao pivot_table. O valor padrão do argumento aggfunc é 'mean'
df_pivoted2 = df_provas2.pivot_table(index='Aluno', columns='Disciplina', values='Objetiva',
                                    aggfunc='max')
df_pivoted2


### Reshaping / Pivoting com Índice Hierárquico
#### Método stack/unstack (Pivotar com índice hierárquico)

In [ ]:
# Considere o seguinte dataframe
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

In [ ]:
# Faça uma operação de unpivoting (stack) com o dataframe data e guarde na variavel result
result = data.stack()
result

In [ ]:
# Faça uma operação de pivoting (unstack) com o dataframe result
result.unstack()